In [1]:
import folium
import requests
import streamlit as st 
from streamlit_folium import folium_static
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely import wkt
import re

In [2]:
# Getting start/dest coordinates
def get_lat_lon(streetname):
    BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'
    response = requests.get(f'{BASE_URL}&street={streetname}&city=Stuttgart')
    data = response.json()
    
    if data:
        lat = data[0].get('lat')
        lon = data[0].get('lon')
        return float(lat), float(lon)
    else:
        # Handle the case where the geocoding service does not return valid data
        return None

In [14]:
# Initializing OSM Graph
def init_osm_graph(city):
    G = ox.graph_from_place(city, network_type='bike')
    return G

G = init_osm_graph('Stuttgart')

In [3]:
# Get fastest route from OSM
def get_osm_route(graph, start_location, dest_location, weight):
     
    start_data = get_lat_lon(start_location)
    dest_data = get_lat_lon(dest_location)

    orig_node = ox.distance.nearest_nodes(graph, start_data[1], start_data[0])
    dest_node = ox.distance.nearest_nodes(graph, dest_data[1], dest_data[0])
    shortest_route = nx.shortest_path(graph, orig_node, dest_node, weight=weight)
    pathDistance = nx.shortest_path_length(graph, orig_node, dest_node, weight=weight)
    
    return shortest_route, pathDistance

In [ ]:
shortest_route, pathDistance = get_osm_route(G, "Königstraße 4", "Tübingerstraße 40", "length")
         
m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
m

In [15]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [28]:
merged_osm = pd.read_csv('OSM\merged_osm.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2845835240.py:1: SyntaxWarning: invalid escape sequence '\m'
  merged_osm = pd.read_csv('OSM\merged_osm.csv')
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2845835240.py:1: DtypeWarning: Columns (7,8,19,20,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_osm = pd.read_csv('OSM\merged_osm.csv')


In [29]:
def db_to_graph(db_nodes, db_edges):
    # nodes = gpd.GeoDataFrame(db_nodes).set_index(["osmid"])
    nodes = db_nodes
    edges = gpd.GeoDataFrame(db_edges).set_index(["u", "v", "key"])
    edges["geometry"] = edges["geometry"].astype(str).apply(lambda x: wkt.loads(x))
    edges = gpd.GeoDataFrame(edges).set_crs("epsg:4326")
    G = ox.graph_from_gdfs(nodes, edges)
    return nodes, edges, G

bike_nodes, bike_edges, G_bike = db_to_graph(nodes, merged_osm)

In [35]:
# Get fastest route from OSM
def get_bike_route(graph, start_location, dest_location, weight):
     
    start_data = get_lat_lon(start_location)
    dest_data = get_lat_lon(dest_location)

    # orig_node = ox.distance.nearest_nodes(graph, start_data[1], start_data[0])
    # dest_node = ox.distance.nearest_nodes(graph, dest_data[1], dest_data[0])
    orig_edge = ox.nearest_edges(graph, start_data[1], start_data[0])
    dest_edge = ox.nearest_edges(graph, dest_data[1], dest_data[0])
    best_route = ox.shortest_path(graph, orig_edge[0], dest_edge[1], weight=weight)
    # best_pathDistance = nx.shortest_path_length(graph, orig_edge, dest_edge, weight=weight)
    
    return best_route #, best_pathDistance

In [36]:
bikeable_route = get_bike_route(G_bike, "Unteraicherstrasse 22", "Schmiedgasse 5", "finalSore_reversed")
         
m = ox.plot_route_folium(G_bike, bikeable_route, tiles='openstreetmap')
m

c:\Users\Zhenzhen\AppData\Local\Programs\Python\Python312\Lib\site-packages\osmnx\routing.py:49: UserWarning: The attribute 'finalSore_reversed' is missing or null on some edges.
  _verify_edge_attribute(G, weight)
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2881458597.py:3: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  m = ox.plot_route_folium(G_bike, bikeable_route, tiles='openstreetmap')


In [37]:
shortest_route, pathDistance = get_osm_route(G, "Unteraicherstrasse 22", "Schmiedgasse 5", "length")
         
m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
m

C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\399718455.py:3: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
